In [ ]:
#| default_exp dmelements

# dmelements
> Operators on density-matrix elements

## Imports -

In [ ]:
#| hide
from fastcore.test import test_close

In [ ]:
#| export
import importlib

from pylgs.imports import *
from pylgs.utilities.nbdev import DictTbl, AttributeTbl
from pylgs.utilities.testing import test_array
from pylgs.utilities.sparse import sparse_kronecker_matrix, sparse_toeplitz, sparse_identity, sparse_diag, sparse
from pylgs.utilities.numpy import sym_range
from pylgs.pymor.parameters import *
from pylgs.pymor.vectorarrays import *
from pylgs.pymor.operators import *
from pylgs.pymor.timestepping import *
from pylgs.pymor.grids import *
from pylgs.pymor.models import *
from pymor.vectorarrays.interface import VectorArray
from pymor.operators.interface import Operator

In [ ]:
#| hide
np.set_printoptions(formatter={'float': lambda x: f'{x:^ 8.2}' if x else f'{0:^ 8}'}, linewidth=140)

## DM elements -

In [ ]:
#| hide
with importlib.resources.path("pylgs.systems.NaD1_Toy", "Flux.mtxn") as path:
    flux = LincombOperator.from_file(path)
dm_elements = flux.assemble().matrix['Density matrix (source)']
dm_element = dm_elements[0].item()
dm_element

'ρ<sub>Re, 3S<sub>1/2</sub>, 3S<sub>1/2</sub></sub>'

In [ ]:
#| export
_dm_element_hfz = re.compile('ρ<sub>([^,]+), \(([^,]+), ([^,]+), ([^)]+)\), \(([^,]+), ([^,]+), ([^)]+)\)</sub>')
_dm_element_toy = re.compile('ρ<sub>([^,]+), ([^,]+), ([^,]+)</sub>')

### parse_dm_element -


In [ ]:
#| export
def parse_dm_element(s):
    if match := _dm_element_hfz.findall(s):
        return match[0][0], match[0][1:4], match[0][4:7] 
    if match := _dm_element_toy.findall(s): 
        return match[0][0], (match[0][1],), (match[0][2],)

In [ ]:
#| hide
[parse_dm_element(e) for e in dm_elements.data]

[('Re', ('3S<sub>1/2</sub>',), ('3S<sub>1/2</sub>',)),
 ('Re', ('3S<sub>1/2</sub>',), ('3P<sub>1/2</sub>',)),
 ('Im', ('3S<sub>1/2</sub>',), ('3P<sub>1/2</sub>',)),
 ('Re', ('3P<sub>1/2</sub>',), ('3P<sub>1/2</sub>',))]

### population_element -


In [ ]:
#| export
def population_element(s):
    parsed_dm = parse_dm_element(s)
    return parsed_dm[1] == parsed_dm[2]

In [ ]:
#| hide
[population_element(e) for e in dm_elements.data]

[True, False, False, True]

### population -

In [ ]:
#| export
# Make this a projection operator onto populations (diagonal elements)
# Then can have sum operators to sum over hyperfine, fine, or all elements
def population(dm_elements):
    return ScaleOperator(
        xr.apply_ufunc(population_element, dm_elements, vectorize=True).astype(bool), 
        name="Population"
    )

In [ ]:
#| hide
population(dm_elements)

ScaleOperator(
    <xarray.DataArray 'Density matrix (source)' (Density matrix (source): 4)> Size: 4B
    array([ True, False, False,  True])
    Coordinates:
      * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
    space=XarrayVectorSpace(
              coords=Coordinates:
                       * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
              name='Density matrix (source)'),
    name='Population')

### level_label -


In [ ]:
#| export
@dispatch
def level_label(expr:str):
    return parse_dm_element(expr)[1][0]

@dispatch
def level_label(expr:DataArray):
    return xr.apply_ufunc(level_label, expr, vectorize=True)

In [ ]:
#| hide
level_label(dm_element)

'3S<sub>1/2</sub>'

In [ ]:
#| hide
level_label(dm_elements)

<xarray.DataArray 'Density matrix (source)' (Density matrix (source): 4)> Size: 256B
array(['3S<sub>1/2</sub>', '3S<sub>1/2</sub>', '3S<sub>1/2</sub>', '3P<sub>1/2</sub>'], dtype='<U16')
Coordinates:
  * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...

### levels -

In [ ]:
#| export
def levels(dm_elements):
    return xr.Variable(
        'Level', 
        list(set().union(xr.apply_ufunc(level_label, dm_elements, vectorize=True).data))
    )

In [ ]:
#| hide
levels(dm_elements)

<xarray.Variable (Level: 2)> Size: 128B
array(['3S<sub>1/2</sub>', '3P<sub>1/2</sub>'], dtype='<U16')

### level_population -


In [ ]:
#| export
def level_population(dm_elements):
    return XarrayMatrixOperator(
        xr.concat([level == level_label(dm_elements) for level in levels(dm_elements)], levels(dm_elements)), 
        name="Population"
    ) * population(dm_elements)

In [ ]:
#| hide
level_population(dm_elements)

ProductOperator(
    [XarrayMatrixOperator(
         <xarray.DataArray 'Density matrix (source)' (Level: 2,
                                                      Density matrix (source): 4)> Size: 8B
         array([[ True,  True,  True, False],
                [False, False, False,  True]])
         Coordinates:
           * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...
           * Level                    (Level) <U16 128B '3S<sub>1/2</sub>' '3P<sub>1/2...,
         source=XarrayVectorSpace(
                    coords=Coordinates:
                             * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...),
         range=XarrayVectorSpace(
                   coords=Coordinates:
                            * Level    (Level) <U16 128B '3S<sub>1/2</sub>' '3P<sub>1/2</sub>',
                   name='Population'),
         name='Population'),
     ScaleOperator(
         <xarray.DataArray 'Density matrix (source)' (Density matrix (source): 4)> Size: 4B
         array([ True, False, False,  True])
         Coordinates:
           * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
         space=XarrayVectorSpace(
                   coords=Coordinates:
                            * Density matrix (source)  (Density matrix (source)) <U50 800B 'ρ<sub>Re, 3...,
                   name='Density matrix (source)'),
         name='Population')])

### sum_over_dm -


In [ ]:
#| export
def sum_over_dm(dm_elements):
    return SumOperator({'Density matrix (source)': dm_elements})

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()